In [1]:
cd "drive/My Drive/Colab Notebooks/DataWorkshop-Matrix/"

/content/drive/My Drive/Colab Notebooks/DataWorkshop-Matrix


In [2]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [3]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

In [0]:
dfx = pd.read_hdf('Data2/car.h5')

In [7]:
dfx.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [15]:
X = dfx[['car_id']].values
y = dfx['price_value'].values
my_model = DummyRegressor()
my_model.fit(X, y)
y_pred = my_model.predict(X)
mae(y, y_pred)

39465.934630440985

In [17]:
[x for x in dfx.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [18]:
dfx['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [19]:
dfx = dfx[dfx['price_currency'] != 'EUR']
dfx.shape

(106290, 155)

In [0]:
suffix = '__cat'
for feat in dfx.columns:
  if isinstance(dfx[feat][0], list): continue
  if 'price' in feat: continue
  if suffix in feat: continue
  dfx[feat + suffix] = dfx[feat].factorize()[0]

In [21]:
cat_feats = [x for x in dfx.columns if suffix in x]
len(cat_feats)

151

In [22]:
X = dfx[cat_feats].values
y = dfx['price_value'].values
my_model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(my_model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [23]:
my_model.fit(X, y)
perm = PermutationImportance(my_model, random_state=0).fit(X, y)

AttributeError: ignored

In [24]:
eli5.show_weights(perm, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat
